In [84]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

## Groupby - basic

* SQL gruopby 명령어와 같음
* split -> apply -> combine
* 과정을 거쳐 연산함

`df.groupby('Team')['Points'].sum()` 
- `('Team')` : 묶음의 기준이 되는 column
- `['Points']` : 적용받는 column
- `.sum()` : 적용받는 연산

In [85]:
# data from: 
ipl_data = {'Team': ['Riders', 'Riders', 'Devils', 'Devils', 'Kings',
         'Kings', 'Kings', 'Kings', 'Riders', 'Royals', 'Royals', 'Riders'],
         'Rank': [1, 2, 2, 3, 3,4 ,1 ,1,2 , 4,1,2],
         'Year': [2014,2015,2014,2015,2014,2015,2016,2017,2016,2014,2015,2017],
         'Points':[876,789,863,673,741,812,756,788,694,701,804,690]}

df = pd.DataFrame(ipl_data)
df

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741
5,Kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701


In [86]:
df.groupby('Team')['Points'].sum()

Team
Devils    1536
Kings     3097
Riders    3049
Royals    1505
Name: Points, dtype: int64

## Hierarchical index

* Groupby 명령의 결과물도 결국은 DataFrame
* 두 개의 column으로 groupby를 할 경우, index가 두 개 생성

In [87]:
# level 0 : 'Team', level 1 : 'Year'
h_index = df.groupby(['Team','Year'])['Points'].sum()
h_index

Team    Year
Devils  2014    863
        2015    673
Kings   2014    741
        2015    812
        2016    756
        2017    788
Riders  2014    876
        2015    789
        2016    694
        2017    690
Royals  2014    701
        2015    804
Name: Points, dtype: int64

In [88]:
h_index.index # MultiIndex 

MultiIndex([('Devils', 2014),
            ('Devils', 2015),
            ( 'Kings', 2014),
            ( 'Kings', 2015),
            ( 'Kings', 2016),
            ( 'Kings', 2017),
            ('Riders', 2014),
            ('Riders', 2015),
            ('Riders', 2016),
            ('Riders', 2017),
            ('Royals', 2014),
            ('Royals', 2015)],
           names=['Team', 'Year'])

* unstack() : Group으로 묶여진 데이터를 matrix 형태로 전환해 줌

In [89]:
h_index.unstack()

Year,2014,2015,2016,2017
Team,,,,
Devils,863.0,673.0,NaN,NaN
Kings,741.0,812.0,756.0,788.0
Riders,876.0,789.0,694.0,690.0
Royals,701.0,804.0,NaN,NaN


* reset_index() : index를 풀어줌

In [90]:
h_index.reset_index()

,Team,Year,Points
0,Devils,2014,863
1,Devils,2015,673
2,Kings,2014,741
3,Kings,2015,812
4,Kings,2016,756
5,Kings,2017,788
6,Riders,2014,876
7,Riders,2015,789
8,Riders,2016,694
9,Riders,2017,690


* swaplevel() : index level을 변경할 수 있음
* sort_index() : level을 기준으로 index를 정렬

In [91]:
h_index.swaplevel().sort_index(level=0)

Year  Team  
2014  Devils    863
      Kings     741
      Riders    876
      Royals    701
2015  Devils    673
      Kings     812
      Riders    789
      Royals    804
2016  Kings     756
      Riders    694
2017  Kings     788
      Riders    690
Name: Points, dtype: int64

* hierarchical index : Series type

In [92]:
type(h_index) 

pandas.core.series.Series

* hierarchical index operation
  * index level을 기준으로 기본 연산 수행 가능

In [93]:
h_index.sum(level=0)

C:\Users\서용득\AppData\Local\Temp\ipykernel_18772\1316823663.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  h_index.sum(level=0)


Team
Devils    1536
Kings     3097
Riders    3049
Royals    1505
Name: Points, dtype: int64

## Groupby - grouped

* Groupby에 의해 split된 상태를 추출 가능함

In [94]:
# Tuple 형태로 그룹의 key값, value값이 추출됨
grouped = df.groupby('Team')

In [95]:
# name 은 string, group은 DataFrame 
for name, group in grouped:
    print(name)
    print(group)

Devils
     Team  Rank  Year  Points
2  Devils     2  2014     863
3  Devils     3  2015     673
Kings
    Team  Rank  Year  Points
4  Kings     3  2014     741
5  Kings     4  2015     812
6  Kings     1  2016     756
7  Kings     1  2017     788
Riders
      Team  Rank  Year  Points
0   Riders     1  2014     876
1   Riders     2  2015     789
8   Riders     2  2016     694
11  Riders     2  2017     690
Royals
      Team  Rank  Year  Points
9   Royals     4  2014     701
10  Royals     1  2015     804


* get_group() : grouped에서 해당 그룹을 추출할 수 있음

In [96]:
grouped.get_group('Kings')

,Team,Rank,Year,Points
4,Kings,3,2014,741
5,Kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788


* Aggregation : 요약된 통계 정보를 추출해 줌

In [97]:
grouped.agg(sum)

,Rank,Year,Points
Team,,,
Devils,5,4029,1536
Kings,9,8062,3097
Riders,7,8062,3049
Royals,5,4029,1505


In [98]:
# 추출되는 건 한 row가 아니라 각 column별로 max값이 추출되는 것임
grouped.agg(max)

,Rank,Year,Points
Team,,,
Devils,3,2015,863
Kings,4,2017,812
Riders,2,2017,876
Royals,4,2015,804


* grouped.describe() : 각 column별로 index들의 통계 정보를 추출해줌

In [99]:
grouped.describe().T

Team               Devils        Kings       Riders       Royals
Rank   count     2.000000     4.000000     4.000000     2.000000
       mean      2.500000     2.250000     1.750000     2.500000
       std       0.707107     1.500000     0.500000     2.121320
       min       2.000000     1.000000     1.000000     1.000000
       25%       2.250000     1.000000     1.750000     1.750000
       50%       2.500000     2.000000     2.000000     2.500000
       75%       2.750000     3.250000     2.000000     3.250000
       max       3.000000     4.000000     2.000000     4.000000
Year   count     2.000000     4.000000     4.000000     2.000000
       mean   2014.500000  2015.500000  2015.500000  2014.500000
       std       0.707107     1.290994     1.290994     0.707107
       min    2014.000000  2014.000000  2014.000000  2014.000000
       25%    2014.250000  2014.750000  2014.750000  2014.250000
       50%    2014.500000  2015.500000  2015.500000  2014.500000
       75%    2014.750000  2016.250000  2016.250000  2014.750000
       max    2015.000000  2017.000000  2017.000000  2015.000000
Points count     2.000000     4.000000     4.000000     2.000000
       mean    768.000000   774.250000   762.250000   752.500000
       std     134.350288    31.899582    88.567771    72.831998
       min     673.000000   741.000000   690.000000   701.000000
       25%     720.500000   752.250000   693.000000   726.750000
       50%     768.000000   772.000000   741.500000   752.500000
       75%     815.500000   794.000000   810.750000   778.250000
       max     863.000000   812.000000   876.000000   804.000000

* Transformation : aggregation과 달리 key값 별로 요약된 정보가 아님, 개별 데이터의 변환을 지원함

In [100]:
# 각 group별로 함수에 따라 값을 변환시켜 출력해줌
score = lambda x: (x-x.mean())/x.std()
grouped.transform(score)

,Rank,Year,Points
0,-1.500000,-1.161895,1.284327
1,0.500000,-0.387298,0.302029
2,-0.707107,-0.707107,0.707107
3,0.707107,0.707107,-0.707107
4,0.500000,-1.161895,-1.042333
5,1.166667,-0.387298,1.183401
6,-0.833333,0.387298,-0.572108
7,-0.833333,1.161895,0.431040
8,0.500000,0.387298,-0.770596
9,0.707107,-0.707107,-0.707107


* filter : 특정 조건으로 데이터를 검색할 때 사용
  * filter 안에는 boolean 조건이 존재해야 함

In [105]:
# 팀별로 3팀 이상인 팀만 보여줘라.
df.groupby('Team').filter(lambda x: len(x) >= 3)

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
4,Kings,3,2014,741
5,Kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
11,Riders,2,2017,690


> 여기서 `filter()` 안의 `x`는 `grouped`를 의미함 (DataFrame type)

In [112]:
df.groupby('Team').filter(lambda x: x['Points'].mean() > 770)

,Team,Rank,Year,Points
4,Kings,3,2014,741
5,Kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
